# SVD  

### Теория  

SVD разложение - разложение прямоугольной матрицы на произведение трех матриц над полем  *R* (или *C*) вида $Matrix = U S V^T$, где $U,V$ ортогональные (унитарные), $S$, матрица с неотрицательными числами на диагонале, идущими по убыванию.  

### Геометрический и физический смысл  

Крайние матрицы отвечают повороту пространства, центральная матрица растяжение вдоль осей. Чем больше сингулярное число - число на диагонале, тем более важным являеется соотвествующий признак.  

### Первое применение  

Впервые применялось в 1998 году в рекомендательных системах для задачи фильтрации данных

### Суть алгоритма в применении к рекомендательным системам  

В задачи с рекомедательнами системами имеем набор матрицу скоринга, которая предсталяет из себя целевые предметы рекоммендации и оценку каждого из объектов, полученные различным образом. В данной задачи рассматривается набор фильмов идентифицирующихся по соответствующему id, и оценка фильмов от 1 до 5 различными пользователями. То есть матрица скоринга R - являтеся матрицей кол-во пользователей $\times$ кол-во фильмов.  
Самое простое применение SVD в данном случае, представляет из себя:  
- Сингулярное разложение матрицы R
- Определение порога на минимальные размер сингулярные значений
- На основе порога выше - выделение некоторого $f \leq rank(R)$ количества оставляемых сингулярных значений.
- Перемножение Матриц $U^{'}, S^{'}, V^{T'}$, где штрих обозначет, что берутся первые f столбцов для матрицы $U$, первые f сингялярных чисел для матрицы $S$, первые f строк для матрицы $V^T$
После применения этого набора действий мы получаем обратно матрицу размера, равного исходной R, но без лишнего фона и признаков.

![How SVD does decomposition (arXiv:2203.11026v1)](SVD.jpg)

### Гипперпараметры  

В данном случае задается один гипперпараметр. Это порог отсечения сингулярных значений, от которого зависит выбор числа f.
Общепринято выбирать порог следующим образом:  

$$\frac{\sum_k^f \sigma^2_{k}}{\sum_i^{rank(R)} \sigma^2_{i}} > threshhold $$

Где $\sigma$ - cингулярные числа, а threshold - минимальное количество информации, которое мы хотим оставить при уменьшении размерности сингулярной матрицы. Обычно ставится равной 95%, однако например в задаче ниже, из-за сильной разреженности матрицы R, выбирались другие значения порога.

## Реализация  
Перед каждой важной группой ячеек написано пояснение, что именно в не происходит и на каком основании.  
В задаче используется датасет https://files.grouplens.org/datasets/movielens/ml-latest-small.zip - набор рейтингов фильмов (9000 фильмов, оцененных 600 пользователями)


## Функции для обработки данных и вычисления RMSE

In [1]:
def pivot(ratings):
    R_matrix = pd.DataFrame()
    R_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')
    R_matrix.rename(columns = dict(zip(R_matrix.columns, ratings.movieId.unique())), inplace = True)
    return R_matrix


def RMSE(R_pred, r):
    rmse = 0
    #R_pred_np = R_pred.to_numpy()
    r_np = r.to_numpy()
    #display(r)
    #display(r_np)
    for i in r_np:
        #print(i)
        #print(R_pred.iloc[int(i[0])][int(i[1])])
        rmse += (i[2] - R_pred[int(i[1])][int(i[0])]) **2

    return (rmse/len(r_np))**0.5

## Подключение модулей, чтение датасета

In [2]:
import pandas as pd
from IPython.display import display
import numpy as np
from sklearn.utils import shuffle

ratings = pd.read_csv("./ml-latest-small/ratings.csv").loc[:,["userId", "movieId", "rating"]]
shuffle(ratings)

,userId,movieId,rating
4244,28,593,2.5
74182,474,3987,3.0
71333,457,2712,3.0
87040,561,102125,2.5
28937,200,19,3.5
...,...,...,...
27376,186,1287,5.0
89863,583,1732,3.5
87181,562,2654,4.5
45635,301,48516,4.5


## Обработка данных  

Здесь мы строим матрицу скоринга R. Заметим, что так как каждый человек не может оценить 9000 фильмов, то матрица R, является достаточно разряженной. Чтобы избавиться от пустот нужно заполнить матрицу некотрым образом. Можно заполнить пропуски, например нулями, однако логичней будет заполнить пропуски для каждого фильма его имеющейся средней оценкой. Сравните матрицу скоринга до заполнения и после:

In [3]:
m, n = ratings.shape

R_matrix_ = pivot(ratings)
R_matrix = R_matrix_.copy()
display(R_matrix)
for cl in R_matrix_.columns:
    mean = R_matrix_[cl].mean()
    R_matrix[cl] = (R_matrix_[cl]).fillna(mean).to_numpy()
display(R_matrix)

movieId,1,3,6,47,50,70,101,110,151,157,...,147662,148166,149011,152372,158721,160341,160527,160836,163937,163981
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


movieId,1,3,6,47,50,70,101,110,151,157,...,147662,148166,149011,152372,158721,160341,160527,160836,163937,163981
userId,,,,,,,,,,,,,,,,,,,,,
1,4.00000,3.431818,4.000000,2.357143,3.071429,4.000000,3.185185,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
2,3.92093,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
3,3.92093,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
4,3.92093,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
5,4.00000,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.50000,3.431818,3.259615,2.357143,3.071429,3.946078,2.500000,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
607,4.00000,3.431818,3.259615,2.357143,3.071429,3.946078,3.185185,2.875,3.125,3.496212,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
608,2.50000,2.000000,2.000000,2.357143,3.071429,3.946078,3.185185,2.875,3.125,4.000000,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0


## Применение SVD  

Так как решение простейшее SVD нужно для сравнения результатов. То оно реализуется не вручную, а с использованием numpy. Здесь применяем разложение, используем критерий отбора сингулярных значений и находим минимальное f, которое нам подходит.  
Заметем, что так как матрица сильно разряженная, и нам пришлось заполнить ее вручную исходя из неких средних соображений, сингялрные числа будут очень похожи и оценка в 95% не будет достигаться. Поэтому в данной задаче ставим нижнее ограничение на f равное 20, что согласуется с дальнейшими вычислениями другими методами, в частости Funk SVD.

In [89]:
print('Rank of the matrix:', np.linalg.matrix_rank(R_matrix))
U, S, V = np.linalg.svd(R_matrix, full_matrices=False)
threshold = 0.95
S2i = S@S
f = len(S)
criteria=1
while criteria >= threshold:
    criteria = S[:f]@S[:f]/S2i
    f-=1
if f < 20: f = 20
print("The f number = ", f)


Rank of the matrix: 610
The f number =  20


## Вычисляем предсказанное значение

In [90]:
Uf = U[:,:f]
Vf = V[:f,:]
Sf = np.diag(S[:f])

R_pred = pd.DataFrame(Uf@Sf@Vf, range(1,611), ratings.movieId.unique())

## Результаты  
Ниже представлены результаты примитивного SVD.   
*Среднее квадратичная ошибка равна* **1.3**  
Значение предсказаний можно сравнить с R_matrix из начала блокнота

In [91]:
display(R_pred)
print("RMSE = ", RMSE(R_pred, ratings))

,1,3,6,47,50,70,101,110,151,157,...,147662,148166,149011,152372,158721,160341,160527,160836,163937,163981
1,4.223314,3.685326,3.609459,2.372336,3.184542,4.143556,3.308495,2.890961,3.149042,3.659126,...,3.506784,3.005815,4.007753,4.007753,3.506784,4.007753,3.506784,3.506784,3.506784,4.007753
2,3.904928,3.443948,3.261369,2.355687,3.072920,3.948800,3.196801,2.877385,3.126003,3.510783,...,3.499985,2.999987,3.999982,3.999982,3.499985,3.999982,3.499985,3.499985,3.499985,3.999982
3,3.810616,3.448173,3.236119,2.334034,3.039455,3.953938,3.214830,2.884585,3.124834,3.529951,...,3.498651,2.998844,3.998459,3.998459,3.498651,3.998459,3.498651,3.498651,3.498651,3.998459
4,3.851166,3.365795,3.198473,2.363422,3.095464,3.823544,3.154685,2.864256,3.152521,3.446632,...,3.499349,2.999442,3.999256,3.999256,3.499349,3.999256,3.499349,3.499349,3.499349,3.999256
5,3.910620,3.424476,3.224231,2.351561,3.080858,3.914411,3.177453,2.877815,3.118496,3.498915,...,3.500956,3.000819,4.001092,4.001092,3.500956,4.001092,3.500956,3.500956,3.500956,4.001092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,4.006354,3.321480,3.227423,2.376160,2.997071,4.104794,3.235117,2.886927,3.081599,3.469586,...,3.506469,3.005545,4.007393,4.007393,3.506469,4.007393,3.506469,3.506469,3.506469,4.007393
607,3.818317,3.471287,3.376988,2.388443,3.175713,4.019260,3.252911,2.877593,3.177281,3.624616,...,3.499844,2.999866,3.999822,3.999822,3.499844,3.999822,3.499844,3.499844,3.499844,3.999822
608,2.113526,2.256271,2.049602,2.407997,2.684466,4.128007,3.011618,2.775405,3.024798,3.628300,...,3.506866,3.005885,4.007847,4.007847,3.506866,4.007847,3.506866,3.506866,3.506866,4.007847
609,3.856964,3.405335,3.220410,2.345881,3.053461,3.930330,3.184224,2.877138,3.126656,3.517278,...,3.500356,3.000305,4.000407,4.000407,3.500356,4.000407,3.500356,3.500356,3.500356,4.000407


RMSE =  1.3115829358543678
